In [8]:
base_dir = "../../../"

#### 1. Importing libraries

In [9]:
from os import listdir, mkdir
from os.path import isdir, isfile, join

In [10]:
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [11]:
from sklearn.metrics import pairwise_distances

#### 2. Function definition

In [12]:
def resize_images(base_dir):
    
    for folder in listdir(base_dir+"Dataset/"):
        for file in listdir(base_dir+"Dataset/"+folder):
            
            image = cv2.imread(base_dir+"Dataset/"+folder+'/'+file)
            converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            shape = converted_image.shape
            add_x = int((224-shape[0])/2)
            add_y = int((224-shape[1])/2)
            distance_map = np.zeros((224,224,3),dtype = np.uint8)
            for i in range(3):
                distance_map[:,:,i] = np.pad(converted_image[:,:,i],((add_x,add_x),(add_y,add_y)),'constant', constant_values = (0))
            
            plt.imsave(base_dir+'Dataset/'+folder+'/'+file, distance_map)
    return

In [13]:
def calculate_distances(coordinates, base_dir, file):
    
    frame = file.split(".")[0]
    folder = frame.split("_")[0]
    
    coordinates = np.array(coordinates)
    distance_matrix = pairwise_distances(coordinates)
       
    if isdir(base_dir+'Dataset/'+folder):
        plt.imsave(base_dir+'Dataset/'+folder+'/'+frame+'.png', distance_matrix)
    else:
        mkdir(base_dir+'Dataset/'+folder)
        plt.imsave(base_dir+'Dataset/'+folder+'/'+frame+'.png', distance_matrix)

    return

In [14]:
def get_coordinates(base_dir):
    
    files = [file for file in listdir(base_dir+"Frames/") if isfile(join(base_dir+"Frames/", file))]
    
    for file in files:
        pdb = open(base_dir+"Frames/"+file, "rt")
        rows = pdb.readlines()
        alpha_carbons = list(filter(lambda l: l[13:15]=="CA", rows))
        pdb.close()
        coordinates = list(map(lambda ca: [float(ca[31:38]), float(ca[39:46]), float(ca[47:54])], alpha_carbons))  
        calculate_distances(coordinates, base_dir, file)
        
    return

In [15]:
get_coordinates(base_dir)

In [16]:
resize_images(base_dir)